In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...

In [6]:
import pymrio
from pymrio.core.mriosystem import IOSystem as IOS
from pymrio.tools.iomath import div0
from pymrio.tools.iofunctions import *
from pymrio.tools.ioparser import *
import pandas as pd
import numpy as np
# import scipy.io
import scipy.sparse as sp 
from scipy.sparse import linalg as spla
import operator
import warnings
import matplotlib.pyplot as plt
from pylab import *
import pickle
from openpyxl import load_workbook
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...
warnings.filterwarnings('ignore')

In [3]:
%%capture
# Unfortunately, the database THEMIS is not open (you can ask to NTNU or thomas.gibon@list.lu) if they can provide it to you
# path_themis = '/media/adrien/dd/adrien/DD/Économie/Données/Themis/'
path_io = '/mnt/dd/adrien/DD/Économie/Données/'
# path_io = '/media/sf_U_DRIVE/Données/'
# years = [2010, 2030, 2050]
# scenarios = ['BL', 'BM']
# scenarios_dlr = ['REF', 'ER', 'ADV'] 
# themis, EROIs_mixes, dlr, EROIs_Mixes = pickle.load(open(path_io+'Themis/EROIs.pkl', 'rb'))
# EROIs_mixes, dlr, EROIs_Mixes = pickle.load(open(path_io+'Themis/EROIs.pkl', 'rb'))

In [7]:
exio3 = parse_exiobase3(path_io+'Exiobase 3.1/exiobase3.4_iot_2011_pxp.zip')

NameError: name 'parse_exiobase3' is not defined

In [26]:
themis = pickle.load(open(path_io+'Themis/themis.pkl', 'rb'))
erois_prices = pickle.load(open(path_io+'Themis/erois_prices.pkl', 'rb'))
# erois_prices_wo_GW = pickle.load(open(path_io+'Themis/erois_prices_wo_GW.pkl', 'rb'))

In [34]:
themis['ER'][2010].find('zinc', 'sectors')

['zinc monosulphate, ZnSO4.H2O, at plant/ RER/ kg',
 'zinc oxide, at plant/ RER/ kg',
 'zinc sulphide, ZnS, at plant/ RER/ kg',
 'cadmium sludge, from zinc electrolysis, at plant/ GLO/ kg',
 'leaching residues, indium rich, from zinc circuit, at smelter/ GLO/ kg',
 'resource correction, PbZn, zinc, negative/ GLO/ kg',
 'resource correction, PbZn, zinc, positive/ GLO/ kg',
 'titanium zinc plate, without pre-weathering, at plant/ DE/ kg',
 'zinc concentrate, at beneficiation/ GLO/ kg',
 'zinc, from combined metal production, at beneficiation/ SE/ kg',
 'zinc, from combined metal production, at refinery/ SE/ kg',
 'zinc, primary, at regional storage/ RER/ kg',
 'zinc coating, coils/ RER/ m2',
 'zinc coating, pieces/ RER/ m2',
 'zinc coating, pieces, adjustment per um/ RER/ m2',
 'disposal, zinc in car shredder residue, 0% water, to municipal incineration/ CH/ kg',
 'mine, gold-silver-zinc-lead-copper/ SE/ unit',
 'Lead, zinc and tin ores and concentrates',
 'Lead, zinc and tin and product